In [ ]:
from cyschoolhousesuite import *
import numpy as np
import os
from simple_cysh import *
# os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management')

In [ ]:
program_df = get_cysh_df('Program__c', ['Id', 'Name'], 
                         rename_id=True)
program_df.rename(columns={'Name': 'Program'},inplace=True)

# get sections, merge in program name, filter for sections of interest
section_df = get_cysh_df('Section__c', ['Id', 'School__c', 'Program__c', 'In_After_School__c'], 
                         rename_id=True, 
#                          where="Active__c = True"
                        )
section_df = section_df.merge(program_df, on='Program__c', how='left')
section_df = section_df.loc[section_df['Program'].isin(['Coaching: Attendance', 'Tutoring: Literacy', 'Tutoring: Math', 'SEL Check In Check Out'])]

stu_sec_df = get_cysh_df('Student_Section__c', ['Id', 'Active__c', 'Section__c', 'Student__c', 'Student_Name__c', 'Student_Grade__c', 'Amount_of_Time__c', 'Intervention_Enrollment_Start_Date__c', 'Enrollment_End_Date__c'],
                         rename_id=True, 
#                          where="Active__c = True"
                        )
stu_sec_df = stu_sec_df.loc[~stu_sec_df['Student_Name__c'].str.contains('DELETE')]
stu_lastnames = get_cysh_df('Student__c', ['Id', 'Student_Last_Name__c'], rename_id=True)
stu_sec_df = stu_sec_df.merge(stu_lastnames, how='left', on='Student__c')

# Get IA's for each student, and include Program to match with sections
stud_ia_df = get_cysh_df('Indicator_Area_Student__c', ['Id', 'Student__c', 'Indicator_Area__c'], 
                         rename_id=True, 
#                          where="Active__c = True"
                        )

ia_df = get_cysh_df('Indicator_Area__c', ['Id', 'Indicator_Area_Type__c'], rename_id=True)
ia_df['Program'] = ia_df['Indicator_Area_Type__c'].map({'Attendance':'Coaching: Attendance',
                                                        'Behavior':'SEL Check In Check Out',
                                                        'Math':'Tutoring: Math',
                                                        'ELA/Literacy':'Tutoring: Literacy'
                                                       })

ia_df = ia_df.merge(program_df, how='left', on='Program')
stud_ia_df = stud_ia_df.merge(ia_df, on='Indicator_Area__c')

school_df = get_cysh_df('Account', ['Id', 'Name'])
school_df.rename(columns={'Id':'School__c', 'Name':'School'}, inplace=True)

In [ ]:
# merges
df = section_df.merge(school_df, how='left', on='School__c')
df = df.merge(stu_sec_df, on='Section__c')
df['Student_Program'] = df['Student__c'] + "_" + df['Program__c']
df.set_index('Student_Program', inplace=True)
df.loc[:,'Intervention_Enrollment_Start_Date__c'] = pd.to_datetime(df['Intervention_Enrollment_Start_Date__c'])
df.loc[:,'Enrollment_End_Date__c'] = df['Enrollment_End_Date__c'].fillna('2018-06-08')
df.loc[:,'Enrollment_End_Date__c'] = pd.to_datetime(df['Enrollment_End_Date__c'])
xsection_start = df.groupby('Student_Program')['Intervention_Enrollment_Start_Date__c'].min()
xsection_end = df.groupby('Student_Program')['Enrollment_End_Date__c'].max()
df.update(xsection_start)
df.update(xsection_end)
df['Days Active'] = (pd.to_datetime(df['Enrollment_End_Date__c']) - pd.to_datetime(df['Intervention_Enrollment_Start_Date__c'])).dt.days

In [ ]:
time_YTD = df.groupby('Student_Program')['Amount_of_Time__c'].sum()
df.update(time_YTD)
# filter out extended learning after ToT sum
df = df.loc[df['In_After_School__c']!='Extended Learning']

df.reset_index(inplace=True)
stud_ia_df['Student_Program'] = stud_ia_df['Student__c'] + "_" + stud_ia_df['Program__c']
df = df.merge(stud_ia_df[['Student_Program', 'Indicator_Area__c', 'Indicator_Area_Type__c']], how='left', on='Student_Program')

df.drop_duplicates('Student_Program', inplace=True)

In [ ]:
# get all assessments with desired fields
assmt_df = get_cysh_df('Assesment__c', ['Id', 'Type__c', 'Date_Administered__c',
                                        'X0_to_300_Scaled_Score__c', 'Student__c',
                                        'Average_Daily_Attendance__c', 'SEL_Composite_T_Score__c'], rename_id=True)

# get assessment type
assmt_types = get_cysh_df('Picklist_Value__c', ['Id', 'Name'])
assmt_types = assmt_types.rename(columns={'Id': 'Type__c', 'Name':'Assessment Type'})
assmt_df = assmt_df.merge(assmt_types, how='left', on='Type__c'); del assmt_df['Type__c']
assmt_df['Score'] = assmt_df[['X0_to_300_Scaled_Score__c', 'Average_Daily_Attendance__c', 'SEL_Composite_T_Score__c']].sum(axis=1)
assmt_df = assmt_df.loc[assmt_df['Score']>0]

In [ ]:
# df.loc[:, 'Assign Indicator Area'] = np.nan

# ATT: if > 56 days in attendance (and has any ADA)
df.loc[df['Program'].str.contains('Attendance')
       & (df['Days Active'] > 56)
       & df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('ADA Tracker'), 'Student__c']), 
      'Assign Indicator Area'] = "Attendance"

# SEL: if >1 day SEL (and has any DESSA)
df.loc[df['Program'].str.contains('SEL')
       & ((df['Days Active'] > 1) | df['Enrollment_End_Date__c'].isnull())
       & df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('DESSA'), 'Student__c']), 
      'Assign Indicator Area'] = "Behavior"

# CP: program time >0 CP (and has NWEA before October)), or grade > 8
df.loc[df['Program'].str.contains('Math')
       & (df['Amount_of_Time__c'] > 0)
       & (df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('MATH')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Assign Indicator Area'] = "Math"
df.loc[df['Program'].str.contains('Literacy')
       & (df['Amount_of_Time__c'] > 0)
       & (df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('ELA')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Assign Indicator Area'] = "ELA/Literacy"

# MISSING ASSESSMENT

# ATT: if > 56 days in attendance (and has any ADA)
df.loc[df['Program'].str.contains('Attendance')
       & (df['Days Active'] > 56)
       & ~df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('ADA Tracker'), 'Student__c']), 
      'Missing Assessment'] = 1

# SEL: if >1 day SEL (and has any DESSA)
df.loc[df['Program'].str.contains('SEL')
       & ((df['Days Active'] > 1))
       & ~df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('DESSA'), 'Student__c']), 
      'Missing Assessment'] = 1

# CP: program time >0 CP (and has NWEA before October)), or grade > 8
df.loc[df['Program'].str.contains('Math')
       & (df['Amount_of_Time__c'] > 0)
       & ~(df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('MATH')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Missing Assessment'] = 1
df.loc[df['Program'].str.contains('Literacy')
       & (df['Amount_of_Time__c'] > 0)
       & ~(df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('ELA')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Missing Assessment'] = 1

# MISSING TIME

# CP: program time >0 CP (and has NWEA before October)), or grade > 8
df.loc[df['Program'].str.contains('Math')
       & ~(df['Amount_of_Time__c'] > 0)
       & (df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('MATH')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Missing Time'] = 1
df.loc[df['Program'].str.contains('Literacy')
       & ~(df['Amount_of_Time__c'] > 0)
       & (df['Student__c'].isin(assmt_df.loc[assmt_df['Assessment Type'].str.contains('ELA')
                                            & (assmt_df['Date_Administered__c'] < '2017-10-01'), 'Student__c'])
          | (df['Student_Grade__c'].astype(int)>8)),
      'Missing Time'] = 1

In [ ]:
counts = df.groupby(['School', 'Program'])['Assign Indicator Area', 'Missing Assessment', 'Missing Time'].count()
counts.reset_index().to_csv(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\Student and IA Counts by School-Program.csv', index=False)

df.groupby(['Program'])['Assign Indicator Area', 'Missing Assessment', 'Missing Time'].count()

In [ ]:
# Students whose IA should never have been assigned:
df.loc[~df['Indicator_Area_Type__c'].isnull() & df['Assign Indicator Area'].isnull()]

In [ ]:
# IA's to Add:

#remove rows with NA indicator area
IA_add = df.loc[df['Indicator_Area_Type__c'].isnull() & ~df['Assign Indicator Area'].isnull()].copy()

IA_add.rename(columns={
    'Student__c':'Student: Student ID',
    'Student_Grade__c':'Student: Grade',
    'Student_Last_Name__c':'Student: Student Last Name',
    'Assign Indicator Area':'Indicator Area',
}, inplace=True)

IA_add = IA_add[['School', 'Student: Student ID', 'Student: Grade', 'Student: Student Last Name', 'Indicator Area']]
IA_add.to_excel(r'C:\Users\City_Year\GitHub\cy-automation-library\cyautomation\cyschoolhouse\input_files\indicator_area_roster.xlsx', index=False)
IA_add.groupby(['School', 'Indicator Area'])['Indicator Area'].count()

In [ ]:
IA_add.head()

In [ ]:
#copy-pasted from Page
from pathlib import Path
#import PageObjects.implementations as scripts
import PageObjects.locators as loc
from seleniumrequests import Firefox
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from time import sleep
import sys

import PageObjects.pages as page

def extract_key():
    """Extract SSO Information from keyfile
    
    One practice I use here is to store my SSO in a keyfile instead of either
    entering it in the cmd prompt on each run or hardcoding it in the program. 
    Keyfile should be formatted like so:
        'descript:cityyear sso/user:aperusse/pass:p@ssw0rd'
    Simply replace the username and password with your credentials and then save
    it as keyfile.txt
    """
    key_file_path = '/'.join([str(Path.home()), 'Desktop/keyfile.txt'])
    with open(key_file_path) as file:
        keys = file.read()
    split_line = keys.split("/")
    entries = [item.split(":")[1] for item in split_line]
    desc, user, pwd = entries
    return user, pwd

class BaseImplementation(object):
    """Base Implementation object
    
    It's an important feature of every implementation that it either take an 
    existing driver or be capable of starting one
    """
    
    gecko_path = str(Path.cwd().parents[1] / 'geckodriver/geckodriver.exe')
    
    def __init__(self, driver=None):
        if driver == None:
            self.driver = Firefox(executable_path=gecko_path)
        else:
            self.driver = driver

class Okta(BaseImplementation):
    """Object for handingling Okta
    
    Wraps all processes for logging into Okta and navigation.
    """
    
    user, pwd = extract_key()
    
    def set_up(self):
        """Navigate to City Year Okta login"""
        self.driver.get("https://cityyear.okta.com")
        
    def enter_credentials(self, username, password):
        """Enter credentials for Okta Login"""
        login_page = page.OktaLoginPage(self.driver)
        assert login_page.page_is_loaded()
        login_page.username = username
        login_page.password = password
        login_page.click_login_button()
    
    def check_logged_in(self):
        """Confirm login"""
        homepage = page.OktaHomePage(self.driver)
        assert homepage.page_is_loaded()
        
    def login(self):
        """Runs all steps to login to Okta"""
        self.set_up()
        self.enter_credentials(self.user, self.pwd)
        self.check_logged_in()
        
    def launch_cyschoolhouse(self):
        """Script for logging into Okta and cyschoolhouse"""
        # Login via okta
        self.login()
        # Nav from Okta home to cyschoolhouse
        Okta = page.OktaHomePage(self.driver)
        assert Okta.page_is_loaded()
        Okta.launch_cyschoolhouse()

class SectionEnrollment(Okta):
    """Implementation script for Section Enrollment"""
    from pandas import read_excel
    
    data = read_excel('input_files/student-enrollment-records.xlsx')
    
    def search_for_a_section(self, section):
        """Should only be used from the cyschoohouse homepage"""
        cysh_home = page.CyshHomePage(self.driver)
        assert cysh_home.page_is_loaded()
        cysh_home.set_search_filter("Sections")
        cysh_home.search_bar = section
        cysh_home.click_search_button()
        
class IndicatorAreaEnrollment(Okta):
    """Implementation object for Indicator Area Enrollment"""
    from pandas import read_excel
    
    data = read_excel('input_files/indicator_area_roster.xlsx')
    student_list = data['Student: Student ID'].unique()
    
    def nav_to_form(self):
        """Initial setup script for IA enrollment.
        
        Goes through login for Okta as well as navigating to the form and 
        ensuring the page is loaded appropriately
        """
        self.launch_cyschoolhouse()
        cysh_home = page.CyshHomePage(self.driver)
        assert cysh_home.page_is_loaded()
        
        self.driver.get("https://c.na24.visual.force.com/apex/IM_Indicator_Areas")
        ia_form = page.CyshIndicatorAreas(self.driver)
        ia_form.wait_for_page_to_load()
    
    def get_student_details(self, student_id):
        """Returns a students details including school, grade, name, and ia list given their id"""
        student_records = self.data[self.data['Student: Student ID'] == student_id]
        school = student_records.School.unique()[0]
        grade = student_records['Student: Grade'].unique()[0]
        name = student_records['Student: Student Last Name'].unique()[0]
        ia_list = student_records['Indicator Area'].values
        return school, grade, name, ia_list
    
    def enroll_student(self, student_id):
        """Handles the enrollment process of all IAs for a single student"""
        ia_form = page.CyshIndicatorAreas(self.driver)
        ia_form.wait_for_page_to_load()
        school, grade, name, ia_list = self.get_student_details(student_id)
        ia_form.select_school(school)
        sleep(1)
        ia_form.select_grade(str(grade))
        sleep(1)
        for ia in ia_list:
            ia_form.name_search = name
            sleep(3)
            self.assign_ias(student_id, ia)
            sleep(2)
        ia_form.save()
    
    def assign_ias(self, student_id, ia):
        """Assign an IA for a single student and indicator"""
        ia_form = page.CyshIndicatorAreas(self.driver)
        ia_form.wait_for_page_to_load()
        ia_form.select_student(student_id)
        sleep(3)
        ia_form.assign_indicator_area(ia)
        
    def enroll_all_students(self):
        """Executes the full IA enrollment"""
        self.nav_to_form()
        self.error_count = 0
        for student_id in self.student_list:
            if self.error_count == 5:
                self.error_count = 0
                self.driver.quit()
                self.driver = Firefox()
                self.nav_to_form()
                
            try:
                self.enroll_student(student_id)
            except TimeoutException:
                print("Timeout Failure on student: {}".format(student_id))
                self.error_count += 1
                
            except StaleElementReferenceException:
                print("Stale Element failure on student: {}".format(student_id))
                self.error_count += 1
                
            except:
                print("Caught a {} error on student: {}".format(sys.exc_info(), student_id))
                self.error_count += 1

In [ ]:
scrpt = IndicatorAreaEnrollment()
scrpt.enroll_all_students()